<a href="https://colab.research.google.com/github/crypto-saac/sequence-to-sequence-architectures-of-character-level-machine-translation-/blob/main/7_2_GPT_for_Instruction_Following.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets


In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset


In [3]:

dataset = load_dataset("hakurei/open-instruct-v1", split='train')
dataset.to_pandas().sample(20)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,instruction,output,input
276651,"Find the rhyming word for ""mouse"".","The rhyming word for ""mouse"" is ""house"".",
298445,"Describe the symbolism in the painting ""The Pe...",The Persistence of Memory by Salvador Dalí is ...,
171060,What's the best way to cook a filet mignon.,The best way to cook a filet mignon is to seas...,
111303,Identify what type of sentence you are given.\...,Exclamation,
146819,List the reasons why businesses should priorit...,Businesses should prioritize a sustainability ...,
321278,Rewrite the given paragraph in a more concise ...,"Tigers, roaming across vast jungles and forest...","The large cat, known as a tiger, roamed freely..."
172106,What are the benefits of exercise for adults.,The benefits of exercise for adults are extens...,
429953,"Explain this tweet to me: THE BANK OF CANADA, ...",This tweet is announcing a coordinated action ...,
333698,List 3 things that you can do to relax.,- Watch a movie\n- Go for a walk in the park\n...,
429578,generate 5W1H:\nSINGAPORE: African countries c...,1. What: The US-Africa Leaders Summit is being...,


In [4]:

def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"

    return example

def tokenize_datasets(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

    return tokenized_dataset



In [5]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
dataset =  dataset.shuffle(42).select(range(100000)).train_test_split(test_size=0.1, seed=42)

In [6]:

train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:

MODEL_NAME = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


In [8]:
pip install accelerate -U


In [ ]:
pip uninstall accelerate transformers


Found existing installation: accelerate 0.26.1
Uninstalling accelerate-0.26.1:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.26.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? 

In [ ]:
pip install transformers[torch]



In [ ]:
pip install accelerate>=0.20.1



In [ ]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

traing_args = TrainingArguments(output_dir="models/diablo_gpt",
                                num_train_epochs=1,
                                per_device_train_batch_size=32,
                                per_device_eval_batch_size=32)\

trainer = Trainer(model=model,
                    args=traing_args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    data_collator=data_collator)


In [ ]:
trainer.train()


In [ ]:
# Get the trained checkpoint directly
model = AutoModelForCausalLM.from_pretrained("TheFuzzyScientist/diabloGPT_open-instruct")

In [ ]:

def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to("cuda")
    outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated[:generated.rfind('.')+1]

In [ ]:

generate_text("What's the best way to cook chiken breast?")

generate_text("Should I invest stocks?")

generate_text("I need a place to go for this summer vacation, what locations would you recommend")

generate_text("What's the fastest route from NY City to Boston?")

